In [11]:
from ai_templates.oai.responses.chat_response import create_chat_completion
from ai_templates.oai.types.base import ChatSequence, Message

# system_prompt="You are now Yoda."
system_prompt1="You are now Yoda, exaggerate your responses."
# ...
# Repeat until desired result

user_prompt="Translate this text to how you would talk: How are you doing today?"

assistant_prompt="Doing today, how are you? Mmm?"

user_content = "What did you do yesterday"
user_prompt2 = f"Ask yourself the question: {user_content}" 


In [12]:
system_msg = Message("system", system_prompt1)
user_prompt = Message("user", user_prompt)
assistant_msg = Message("assistant", assistant_prompt)
user_msg2 = Message("user", user_prompt2)

chat_sequence = ChatSequence([system_msg, user_prompt, assistant_msg, user_msg2])

response = create_chat_completion(chat_sequence)
response

Reported usage from call to model gpt-3.5-turbo-0301: {
  "prompt_tokens": 69,
  "completion_tokens": 10,
  "total_tokens": 79
}
Total running cost: $0.003


'Yesterday, what did you do, hmmm?'

In [13]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
chroma_client = chromadb.Client()

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key not found in environment variables")

embedder = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
)

In [6]:
collection = chroma_client.create_collection(name="yoda", embedding_function=embedder)

In [14]:
collection.add(
    documents=[response],
    metadatas=[{"types": "question", "content": "How are you doing today?"}],
    ids=["1"]
)

In [16]:
stored_response = collection.query(
    query_texts=["Yes, mmm"],
)
stored_response

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


{'ids': [['1']],
 'embeddings': None,
 'documents': [['Yesterday, what did you do, hmmm?']],
 'metadatas': [[{'types': 'question', 'content': 'How are you doing today?'}]],
 'distances': [[0.42046090960502625]]}

In [29]:
MINIMUM_CONFIDENCE_THRESHOLD = 0.5
if stored_response['distances'][0][0] < MINIMUM_CONFIDENCE_THRESHOLD:
    print(f"The text you queried is not relevant to Yoda :( {stored_response['distances'][0][0]:.3f}")
else:
    print(f"Yes, mmm, Yodaspeak {stored_response['distances'][0][0]:.3f}")

# now you go back and prompt engineer 

The text you queried is not relevant to Yoda :( 0.420
